In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [3]:
data = pd.read_csv("C:\\Users\\USER\\Downloads\\Bengaluru_House_Data.csv")

In [6]:
df = data.copy()

In [7]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [8]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [12]:
df.duplicated().sum()

np.int64(529)

In [13]:
df.dtypes

area_type        object
availability     object
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

In [15]:
df.describe()

,bath,balcony,price
count,13247.000000,12711.000000,13320.000000
mean,2.692610,1.584376,112.565627
std,1.341458,0.817263,148.971674
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [25]:
df['bath'].isnull().sum()

np.int64(0)

In [23]:
df['balcony'].isnull().sum()

np.int64(0)

In [21]:
df['price'].isnull().sum()

np.int64(0)

In [29]:
df['price'] = df['price'].fillna(0).astype(int)

In [30]:
df['balcony'] = df['balcony'].fillna(0).astype(int)

In [31]:
df['bath'] = df['bath'].fillna(0).astype(int)

In [32]:
# Select only object (string) columns
categorical_cols = df.select_dtypes(include=['object']).columns
print("Categorical Columns:", categorical_cols)


Categorical Columns: Index([], dtype='object')


In [33]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in categorical_cols:
    df[col] = le.fit_transform(df[col].astype(str))


In [34]:
df.dtypes

area_type       int64
availability    int64
location        int64
size            int64
society         int64
total_sqft      int64
bath            int64
balcony         int64
price           int64
dtype: object

In [35]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,3,40,419,13,464,70,2,1,39
1,2,80,317,19,2439,1288,5,3,120
2,0,80,1179,16,2688,514,2,3,62
3,3,80,757,16,2186,602,3,1,95
4,3,80,716,13,2688,239,2,1,51


In [37]:
df.isnull().sum()

area_type       0
availability    0
location        0
size            0
society         0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64

In [38]:
df.corr()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
area_type,1.000000,-0.111421,0.033184,-0.096801,-0.149063,-0.117734,-0.101883,0.098609,-0.057560
availability,-0.111421,1.000000,0.002804,0.090900,0.016054,0.027519,0.070081,0.083836,0.031433
location,0.033184,0.002804,1.000000,0.006607,-0.050931,-0.000023,0.003266,0.002623,0.011419
size,-0.096801,0.090900,0.006607,1.000000,0.115643,-0.063268,0.653555,0.154994,0.331921
society,-0.149063,0.016054,-0.050931,0.115643,1.000000,0.035172,0.131431,-0.066642,0.058392
total_sqft,-0.117734,0.027519,-0.000023,-0.063268,0.035172,1.000000,0.169503,-0.096404,0.230793
bath,-0.101883,0.070081,0.003266,0.653555,0.131431,0.169503,1.000000,0.088754,0.448268
balcony,0.098609,0.083836,0.002623,0.154994,-0.066642,-0.096404,0.088754,1.000000,0.019746
price,-0.057560,0.031433,0.011419,0.331921,0.058392,0.230793,0.448268,0.019746,1.000000


In [39]:
df = df.drop(['area_type','society','availability'], axis=1)

In [40]:
df

,location,size,total_sqft,bath,balcony,price
0,419,13,70,2,1,39
1,317,19,1288,5,3,120
2,1179,16,514,2,3,62
3,757,16,602,3,1,95
4,716,13,239,2,1,51
...,...,...,...,...,...,...
13315,1252,22,1499,4,0,231
13316,1004,18,1535,5,0,400
13317,972,13,172,2,1,60
13318,907,18,1680,4,1,488


In [41]:
X = df[['location','size','total_sqft','bath']]   # Features
y = df['price']                                   # Target


In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [43]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1,1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1,1)).ravel()

In [44]:
svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_model.fit(X_train_scaled, y_train_scaled)

SVR(C=100, gamma=0.1)

In [45]:
y_pred_scaled = svr_model.predict(X_test_scaled)

# Inverse transform predictions
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1,1))

# Metrics
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2 Score:", r2_score(y_test, y_pred))

RMSE: 121.73058017499193
R2 Score: 0.3043910584125057
